In [20]:
import psycopg2
import math
from random import randint


In [21]:
with open("USA_states.txt") as f:
    states = f.readlines()
states = [x.strip() for x in states]

with open("state_codes.txt") as f:
    state_codes = f.readlines()
state_codes = [x.strip() for x in state_codes]

# create sate table
state_table = []
for state in states:
    state_info = {"state_name": state}
    state_table.append(state_info) 
    
state_table_t = tuple(state_table)
#state_table_t

In [22]:
#Create entries for customer table customer name = State_code+number(0:10000)
customer_list = []
customer_table = []
#CUST_SIZE = 10
CUST_SIZE = 10000
padding = int(math.log10(CUST_SIZE))
for i in range (CUST_SIZE):
    for state_code in state_codes:
        customer_list.append(state_code+str(i).zfill(padding))
        cust_info = {
            "customer_name" : state_code+str(i).zfill(padding),
            "state_id"      : randint(2,len(state_codes))
        }
        customer_table.append(cust_info)

customer_table_t = tuple(customer_table)

In [23]:
#create entries for sales category table
category_table = []
category_list = []
#CAT_SIZE = 10
CAT_SIZE = 30
padding = int(math.log10(CAT_SIZE))
for i in range (CAT_SIZE):
    category_list.append("CAT"+str(i).zfill(padding))
    cat_info = {
        "category_name" : "CAT"+str(i).zfill(padding),
        "category_desc" : "Product Category "+str(i).zfill(padding)
    }
    category_table.append(cat_info)

category_table_t = tuple(category_table)

In [24]:
len(category_table_t)

30

In [25]:
#create sales table. 1000 products in each category
product_table = []
product_list = []
#PROD_SIZE = 10
PROD_SIZE = 20
padding = int(math.log10(PROD_SIZE))
for cat in category_list:
    for i in range (PROD_SIZE):
        product_list.append("Prod"+str(i).zfill(padding)+cat)
        prod_info = {
            "product_name" : "Prod"+str(i).zfill(padding)+cat,
            "category_id" : category_list.index(cat)+1,
            "list_price"   : float(category_list.index(cat)+10)
        }
        product_table.append(prod_info)

product_table_t = tuple(product_table)

In [26]:
len(product_list)

600

In [27]:
#create sale database
sale_table = []
sale_list = []

for i in range(100000):
    sale_info = {
        "customer_id" : randint(2, CUST_SIZE),
        "product_id"  :  randint(2, PROD_SIZE),
        "quantity"    : randint(1,9),
        "price"       : float(randint(1,50)+10)
    }
    sale_table.append(sale_info)

sale_table_t = tuple(sale_table)

In [28]:
len(sale_table_t)

100000

In [29]:
sale_table_t[10]

{'customer_id': 1919, 'price': 54.0, 'product_id': 15, 'quantity': 9}

In [30]:
# create a connection to database. make sure that the db name and password are correct
try:
    conn = psycopg2.connect("dbname='dse201' user='postgres' host='localhost' password='postgres'")
except:
    print "unable to connect to the database"
    
    
conn.set_session(autocommit = True)
cur = conn.cursor()

In [31]:
cur.executemany("""INSERT INTO sales.state(state_name) VALUES (%(state_name)s)""", state_table_t)

In [32]:
cur.executemany("""INSERT INTO sales.customer(customer_name,state_id) VALUES (%(customer_name)s, %(state_id)s)""",
                customer_table_t)


In [33]:
cur.executemany("""INSERT INTO sales.category(category_name,category_desc) VALUES (%(category_name)s, %(category_desc)s)""",
                category_table_t)

In [34]:
cur.executemany("""INSERT INTO sales.product(product_name,category_id,list_price) VALUES (%(product_name)s, %(category_id)s, %(list_price)s)""",
                product_table_t)

In [35]:
cur.executemany("""INSERT INTO sales.sale(customer_id,product_id,quantity,price) VALUES (%(customer_id)s, %(product_id)s, %(quantity)s, %(price)s)""",
                sale_table_t)

In [36]:
try:
    cur.execute("""SELECT * from sales.sale limit 10""")
except:
    print "Failed SELECT from sales.sale"

rows = cur.fetchall()
for row in rows:
    print "   ", row

    (1, 1138, 15, 6, Decimal('60.0'))
    (2, 7415, 8, 3, Decimal('34.0'))
    (3, 5856, 2, 2, Decimal('50.0'))
    (4, 9119, 12, 6, Decimal('57.0'))
    (5, 3163, 9, 9, Decimal('59.0'))
    (6, 1067, 5, 2, Decimal('32.0'))
    (7, 219, 13, 9, Decimal('42.0'))
    (8, 7427, 18, 3, Decimal('42.0'))
    (9, 6018, 6, 1, Decimal('15.0'))
    (10, 5279, 7, 3, Decimal('22.0'))


In [37]:

try:
    cur.execute("""SELECT * from sales.customer limit 5""")
except:
    print "Failed SELECT from sales.customer"

rows = cur.fetchall()
for row in rows:
    print "   ", row

    (1, 'AL0000', 11)
    (2, 'AK0000', 33)
    (3, 'AZ0000', 36)
    (4, 'AR0000', 43)
    (5, 'CA0000', 17)


In [38]:
#close the connection to db
conn.close()